In [3]:
import cv2
import os
import numpy as np
import pandas as pd

In [4]:
import xlrd   #引入库
wb=xlrd.open_workbook(r"C:\Users\CANDY\Desktop\广告编码1-200.xlsx")
sheet_names=wb.sheet_names() 
sheet=wb.sheet_by_index(1)  
bvs=sheet.col_values(7)
titleyuanbens=sheet.col_values(2)
begins=sheet.col_values(3)
ends=sheet.col_values(4)

In [ ]:
#titleyuanbens

In [ ]:
titles=[]
from bilibili_api import video
for i in range(1,len(bvs)):
    try:#print(i)
        bvid=bvs[i]
        video_info=video.get_video_info(bvid=bvid)
        titleyuanben = video_info["title"]
        #print(video_info["title"])
        titles.append(titleyuanben)
    except:
        titles.append('')

In [ ]:
import pandas as pd
df1 = pd.DataFrame()
df1['title']=titles
df1.to_excel(r"C:\Users\CANDY\Desktop\testtitle.xls")

In [5]:
#清除储存文件中原有的截图
def del_file(path):
    for i in os.listdir(path):
        # 如果存在文件夹进行递归
        if os.path.isdir(os.path.join(path, i)):
            del_file(os.path.join(path, i))
        # 如果是文件进行删除
        elif os.path.isfile:
            os.remove(os.path.join(path, i))

In [6]:
#遍历所有截图计算平均HSI
def avghsv(path):
    #初始化
    hs=[]
    ss=[]
    vs=[]
    img_paths=[]
    dir_paths=os.listdir(path)
    #得到每个图像的路径
    for dir_path in dir_paths:
        img_path=os.path.join(path,dir_path)
        img_paths.append(img_path)
    for img in img_paths:
        if img.endswith('.jpg'): 
            #该图片的h,s,v平均值
            pict=cv2.imread(img)
            hsv=cv2.cvtColor(pict,cv2.COLOR_BGR2HSV)#转化成hsv空间
            h=hsv[:,:,0]
            s=hsv[:,:,1]
            v=hsv[:,:,2]#像素矩阵最后一维012对应hsv三个字母
            h_mean=np.mean(h)
            s_mean=np.mean(s)
            v_mean=np.mean(v)
            hs.append(h_mean)
            ss.append(s_mean)
            vs.append(v_mean)
    #求这几个图片的平均hsv
    hs_mean=np.mean(hs)
    ss_mean=np.mean(ss)
    vs_mean=np.mean(vs)
    
    return hs_mean,ss_mean,vs_mean    

In [7]:
#读入视频路径
inputdir=r"D:\CRTubeGet Downloaded"
#输出图片路径
outputdir=r"C:\Users\CANDY\Desktop\video_to_ppt"

In [8]:
import cv2
import os
def process_video(start,end,videodir,outputdir):
    #清除储存文件中原有的截图
    del_file(outputdir)
    # 定义视频文件路径和保存图像文件路径
    video_path =videodir
    save_path =outputdir
    
    if not os.path.exists(save_path):
        os.makedirs(save_path)
     # 定义起始和结束时间，以及帧数间隔
    if start==0:
        start_time =start # 单位为秒
    else:
        start_time=start-2
    end_time =end# 单位为秒

    # 打开视频文件
    cap = cv2.VideoCapture(video_path) 
    # 从指定的时间开始读取视频帧
    cap.set(cv2.CAP_PROP_POS_MSEC, start_time *1000) 
    # 定义计数器变量
    i = 0 
    # 读取视频帧，并保存每个间隔的帧为图像文件
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            # 判断当前时间是否在指定的时间段内
            current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000
            if current_time >= start_time and i<=50:
                print(current_time)
                # 保存当前帧为图像文件
                out_file_name = str(i) + ".jpg"
                out_file_path = os.path.join(save_path, out_file_name)
                cv2.imwrite(out_file_path, frame)
                i += 1
                print("processed frame:", i-1)
            if current_time >= end_time:
                break
        else:
            break
    # 关闭视频文件
    cap.release()
    return i-1

In [9]:
# Hash值对比
def cmpHash(hash1, hash2,shape=(10,10)):
    n = 0
    # hash长度不同则返回-1代表传参出错
    if len(hash1)!=len(hash2):
        return -1
    # 遍历判断
    for i in range(len(hash1)):
        # 相等则n计数+1，n最终为相似度
        if hash1[i] == hash2[i]:
            n = n + 1
    return n/(shape[0]*shape[1])
# 感知哈希算法(pHash)
def pHash(img,shape=(10,10)):
    # 缩放32*32
    img = cv2.resize(img, (32, 32))  # , interpolation=cv2.INTER_CUBIC

    # 转换为灰度图
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # 将灰度图转为浮点型，再进行dct变换
    dct = cv2.dct(np.float32(gray))
    # opencv实现的掩码操作
    dct_roi = dct[0:10, 0:10]

    hash = []
    avreage = np.mean(dct_roi)
    for i in range(dct_roi.shape[0]):
        for j in range(dct_roi.shape[1]):
            if dct_roi[i, j] > avreage:
                hash.append(1)
            else:
                hash.append(0)
    return hash

In [11]:
import numpy as np
import jieba
#读取停用词

def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r+',encoding='utf-8').readlines()]
    return stopwords

# 加载停用词
stopwords = stopwordslist(r'C:\Users\CANDY\代码\停用词.txt')

def cosine_similarity(sentence1: str, sentence2: str) -> float:
    """
    :param sentence1: s
    :param sentence2:
    :return: 两句文本的相识度
    """
    seg1 = [word for word in jieba.cut(sentence1) if word not in stopwords]
    seg2 = [word for word in jieba.cut(sentence2) if word not in stopwords]
    word_list = list(set([word for word in seg1 + seg2]))#建立词库
    word_count_vec_1 = []
    word_count_vec_2 = []
    for word in word_list:
        word_count_vec_1.append(seg1.count(word))#文本1统计在词典里出现词的次数
        word_count_vec_2.append(seg2.count(word))#文本2统计在词典里出现词的次数

    vec_1 = np.array(word_count_vec_1)
    vec_2 = np.array(word_count_vec_2)
    #余弦公式

    num = vec_1.dot(vec_2.T)
    denom = np.linalg.norm(vec_1) * np.linalg.norm(vec_2)
    cos = num / denom
    sim = 0.5 + 0.5 * cos

    return sim

In [ ]:
#修改路径
import os
import re
import time
  
"""对指定目录下的所有文件进行有选择的修改名称"""
def ReFileName(dirPath,pattern):
    """
    :param dirPath: 文件夹路径
    :param pattern: 正则匹配模式
    :return:
    """
    # 对目录下的文件进行遍历
    #print(os.listdir(dirPath))
    for file in os.listdir(dirPath):
        #print(file)
        # 判断是否是文件
        if os.path.isfile(os.path.join(dirPath, file)) == True:
            # 用正则匹配，去掉不需要的词
            newName = re.sub(pattern, "", file)
            print('newName',newName)
            # 设置新文件名
            newFilename = file.replace(file, newName)
            print('newFilename',newFilename)
            # 重命名
            try:
                os.rename(os.path.join(dirPath, file), os.path.join(dirPath, newFilename))
            except:
                pass
    print("文件名已统一修改成功")  
   
if __name__ == '__main__':
    timeStart = time.time()
    Path=r'D:\CRTubeGet Downloaded'
    
#   pattern = re.compile(r'\[{1}(.+)]\.')
    pattern = re.compile(r'-{1}(.+)zh')
    ReFileName(Path,pattern)
    timeEnd = time.time()
    print("程序走了%d秒"%(timeEnd-timeStart))

In [13]:
import re,openpyxl
def docsimilarity(path,begin_time,end_time):
    print(begin_time,end_time)
    path=path
    begintime=begin_time
    endtime=end_time
    new = []
    zimubegins=[]
    try:
        
        with open(path, encoding="utf-8-sig") as f:
            for sub in f.readlines():
                if sub[:3]== '00:':
                    begin_min= int(sub.split(':')[1])
                    begin_sec= int(sub.split('-')[0].split(',')[0].split(':')[2])
                    begin=begin_min*60+begin_sec
                    zimubegins.append(begin)
                #print(sub[0],sub[-1])
                #if sub[0] not in list('\n0123456789') and sub[-2] not in list('0123456789'):
                if sub[0] not in list('\n0123456789') and sub[-1] not in list('0123456789'):
                    new.append(sub)
        news = [ele.strip() for ele in new]
        #print(news)
        zimu=pd.DataFrame()
        n=len(news)
        zimubegins=zimubegins[:n]
        zimu['begin']=zimubegins
        zimu['new']=news
        
        ad=''
        noad=''

        for i in range(len(zimu)):
            if begintime<=zimu.begin[i]<=endtime:
                docsimilarity=1
                ad=ad+','+zimu.new[i]
            else:
                noad=noad+','+zimu.new[i]
        #print('ad',ad)
        #print('noad',noad)
        if len(noad)==0 and len(ad)!=0:
            docsimilarity=1
        elif len(ad)==0 and len(noad)!=0:
            docsimilarity=0
        else:
            docsimilarity=cosine_similarity(ad,noad)  
    except:
        pass

    return docsimilarity    

In [ ]:
file_list=os.listdir(inputdir)
error=[]
video=pd.DataFrame()
#文件夹下有多少视频
for j in range(92,93):
    print('###')
    titleyuanben=titleyuanbens[j]
    start_time=int(begins[j])
    end_time=int(ends[j])
    print(titleyuanben)
    title=titleyuanben
    h=0
    v=0
    s=0
    similarity=0
    for i in range(len(file_list)):
        inputdir_title=file_list[i].replace('.mp4','')
        #title=inputdir_title.replace('.mp4','')
        if inputdir_title==titleyuanben:
            try:
                print("Step 1:对视频截帧")
                videodir=os.path.join(inputdir,inputdir_title)+'.mp4'
                print(videodir)
                n=process_video(start_time,end_time,videodir,outputdir) 
                print(n)

                print("Step 2:")
                print("计算图片相似度")
                if end_time==0:
                    similarity=1.0
                else:
                    img1 = cv2.imread(r"C:\Users\CANDY\Desktop\video_to_ppt\0.jpg")  
                    img2 = cv2.imread(r"C:\Users\CANDY\Desktop\video_to_ppt\%d.jpg"%(n))
                    hash1 = pHash(img1)
                    hash2 = pHash(img2)
                    similarity=cmpHash(hash1, hash2)
                print(similarity)
                print("Step 3:")
                print("计算广告平均HSV")
                h,s,v=avghsv(outputdir)
                print('h平均值为%s,\ns平均值为%s,\nv平均值为%s'%(h,s,v))

                #item={'title':title,'h':h,'s':s,'v':v,'similarity':similarity}
                #video=video.append(item,ignore_index=True)
            except:
                inputdir_title=file_list[i]
                error.append(inputdir_title)
    item={'title':title,'h':h,'s':s,'v':v,'picsimilarity':similarity}
    video=video.append(item,ignore_index=True)

In [ ]:
#文字相似度
file_list=os.listdir(inputdir)
video1=pd.DataFrame()
#文件夹下有多少视频
for j in range(92,93):
    print('###')
    titleyuanben=titleyuanbens[j]
    start_time=int(begins[j])
    end_time=int(ends[j])
    print(titleyuanben)
    title=titleyuanben
    similarity=111
    for i in range(len(file_list)):
        inputdir_title=file_list[i].replace('.srt','')
        
        #title=inputdir_title.replace('.mp4','')
        if inputdir_title==titleyuanben:
            if end_time==0:
                similarity=0
            else:
                print('计算文本相似度')    
                srtdir=os.path.join(inputdir,inputdir_title)+'.srt'
                print(srtdir)
                similarity=docsimilarity(srtdir,begins[j],ends[j])
                print(similarity)
    item={'title':titleyuanben,'docsimilarity':similarity}
    video1=video1.append(item,ignore_index=True)

In [42]:
video.to_csv(r'C:\Users\CANDY\Desktop\video.csv',encoding="utf_8_sig")